In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

# Rolling mean by Pandas DataFrame Groups

Let say in a Pandas DataFrame we have a date column, several categorical columns and a target column.
We would like to compute a rolling mean of the target, for one or two categories, and for a specific date window.

First we will create some sample data, then we review the normal Pandas DataFrame rolling method, and finally we will try to do rolling by groups.

## Sample data

In [3]:
# We will make a DataFrame of this number of rows
n_row = 20_000_000
n_date = 365

365

In [4]:
# Set the dates of interest
start_date = pd.Timestamp(year=2022, month=1, day=1)
all_dates_full = pd.date_range(start=start_date, periods=n_date)
all_dates = np.random.choice(all_dates_full, size=n_date*4//5, replace=False)
all_dates.sort()
pd.Series(all_dates)

0     2022-01-01
1     2022-01-02
2     2022-01-04
3     2022-01-05
4     2022-01-06
         ...    
287   2022-12-26
288   2022-12-27
289   2022-12-28
290   2022-12-30
291   2022-12-31
Length: 292, dtype: datetime64[ns]

In [5]:
# A category (group)
groups = list("ABC")

['A', 'B', 'C']

In [6]:
# The target (response)
targets = [0, 1]

[0, 1]

In [7]:
df = pd.DataFrame(dict(
    ts=pd.Series(np.random.choice(all_dates, n_row)),
    gr=pd.Series(np.random.choice(groups, n_row)),
    y=pd.Series(np.random.choice(targets, n_row)),
)).sort_values(["ts", "gr"])

,ts,gr,y
11,2022-01-01,A,1
1266,2022-01-01,A,1
2019,2022-01-01,A,0
5166,2022-01-01,A,1
5387,2022-01-01,A,1
...,...,...,...
19997170,2022-12-31,C,0
19997608,2022-12-31,C,0
19998292,2022-12-31,C,0
19999115,2022-12-31,C,1


## Normal Pandas rolling

Now look at how Pandas can do rolling.

In [8]:
df.set_index("ts").rolling("2D")["y"].mean()

ts
2022-01-01    1.000000
2022-01-01    1.000000
2022-01-01    0.666667
2022-01-01    0.750000
2022-01-01    0.800000
                ...   
2022-12-31    0.501974
2022-12-31    0.501970
2022-12-31    0.501966
2022-12-31    0.501970
2022-12-31    0.501973
Name: y, Length: 20000000, dtype: float64

## Roll by group

Now let's roll in a group.

In [9]:
df_gr = df.groupby(["gr", "ts"])["y"].agg(["sum", "count"])

sum  count
gr ts                      
A  2022-01-01  11325  22813
   2022-01-02  11501  22905
   2022-01-04  11271  22826
   2022-01-05  11373  22824
   2022-01-06  11385  22668
...              ...    ...
C  2022-12-26  11119  22600
   2022-12-27  11610  23242
   2022-12-28  11412  22788
   2022-12-30  11368  22877
   2022-12-31  11430  22844

[876 rows x 2 columns]

As you can see above, it is easy and fast to do normal groupby aggregation.
Now, to compute the rolling mean for group A for 1 week, we have to consider both the mean for each day and the corresponding count.

In [10]:
df_gr.loc["A"]

,sum,count
ts,,
2022-01-01,11325,22813
2022-01-02,11501,22905
2022-01-04,11271,22826
2022-01-05,11373,22824
2022-01-06,11385,22668
...,...,...
2022-12-26,11355,22679
2022-12-27,11266,22572
2022-12-28,11599,22922


In [11]:
dfa = df_gr.loc["A"].reindex(all_dates_full) #.fillna(0)

,sum,count
2022-01-01,11325.0,22813.0
2022-01-02,11501.0,22905.0
2022-01-03,NaN,NaN
2022-01-04,11271.0,22826.0
2022-01-05,11373.0,22824.0
...,...,...
2022-12-27,11266.0,22572.0
2022-12-28,11599.0,22922.0
2022-12-29,NaN,NaN
2022-12-30,11545.0,22799.0


In [12]:
dfa_roll = dfa.rolling("2D").sum().rename(columns=dict(sum="sum_roll", count="count_roll"))

,sum_roll,count_roll
2022-01-01,11325.0,22813.0
2022-01-02,22826.0,45718.0
2022-01-03,11501.0,22905.0
2022-01-04,11271.0,22826.0
2022-01-05,22644.0,45650.0
...,...,...
2022-12-27,22621.0,45251.0
2022-12-28,22865.0,45494.0
2022-12-29,11599.0,22922.0
2022-12-30,11545.0,22799.0


In [13]:
dfa_roll["mean_roll"] = (dfa_roll["sum_roll"]/dfa_roll.loc[:, "count_roll"]) #.fillna(0)

In [14]:
pd.concat([dfa, dfa_roll], axis=1)

,sum,count,sum_roll,count_roll,mean_roll
2022-01-01,11325.0,22813.0,11325.0,22813.0,0.496427
2022-01-02,11501.0,22905.0,22826.0,45718.0,0.499278
2022-01-03,NaN,NaN,11501.0,22905.0,0.502117
2022-01-04,11271.0,22826.0,11271.0,22826.0,0.493779
2022-01-05,11373.0,22824.0,22644.0,45650.0,0.496035
...,...,...,...,...,...
2022-12-27,11266.0,22572.0,22621.0,45251.0,0.499901
2022-12-28,11599.0,22922.0,22865.0,45494.0,0.502594
2022-12-29,NaN,NaN,11599.0,22922.0,0.506020
2022-12-30,11545.0,22799.0,11545.0,22799.0,0.506382


We may need to shift the data to reflect the time gap to data availability.

In [15]:
dfa_roll["mean_roll"].shift(periods=1)

2022-01-01         NaN
2022-01-02    0.496427
2022-01-03    0.499278
2022-01-04    0.502117
2022-01-05    0.493779
                ...   
2022-12-27    0.500683
2022-12-28    0.499901
2022-12-29    0.502594
2022-12-30    0.506020
2022-12-31    0.506382
Freq: D, Name: mean_roll, Length: 365, dtype: float64

We have computed the mean of the target variable y for data in group A by rolling over a time window. 
If the cardinality of the group is high, i.e. there are many groups, then the computation above has to be done for each group,
and it is time consuming. 